## 数据清洗

In [2]:
import pandas as pd
import re
from opencc import OpenCC

def clean_abstract(text):
    """清洗abstract字段"""
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # 1. 移除无用标签：【背景】【目的】【方法】【结果】【结论】等
    pattern_tags = r'【[^】]*】|<[^>]*>|〔[^〕]*〕'
    text = re.sub(pattern_tags, '', text)
    
    # 2. 移除特殊字符和异常字符
    # 保留中文、英文、数字、基本标点
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\s，。、；：""''（）！？,.;:!?\-—…·\n]', '', text)
    
    # 3. 移除多余空白
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    # 4. 繁简转换（繁体转简体）
    cc = OpenCC('t2s')  # traditional to simplified
    text = cc.convert(text)
    
    # 5. 截断到500字
    if len(text) > 500:
        text = text[:500]
    
    return text

def process_excel(input_file, output_file):
    """处理Excel文件"""
    # 读取Excel
    df = pd.read_excel(input_file)
    
    # 检查是否存在abstract字段
    if 'abstract' not in df.columns:
        raise ValueError("Excel文件中未找到'abstract'字段")
    
    # 清洗abstract字段
    df['abstract'] = df['abstract'].apply(clean_abstract)
    
    # 保存结果
    df.to_excel(output_file, index=False)
    
    print(f"处理完成！")
    print(f"总行数: {len(df)}")
    print(f"有效abstract数: {(df['abstract'] != '').sum()}")
    print(f"输出文件: {output_file}")

if __name__ == "__main__":
    input_file = "data/全部元数据.xlsx"
    output_file = "output/清洗后元数据.xlsx"
    
    process_excel(input_file, output_file)

处理完成！
总行数: 10423
有效abstract数: 10423
输出文件: output/清洗后元数据.xlsx


## 通用信息提取

In [4]:
import pandas as pd
import sys

def create_info_field(row):
    """创建info字段内容"""
    info_parts = []
    
    # 添加abstract
    abstract = row.get('abstract', '')
    if pd.notna(abstract) and str(abstract).strip():
        info_parts.append(f"abstract: {abstract}")
    
    # 添加date
    date = row.get('date', '')
    if pd.notna(date) and str(date).strip():
        info_parts.append(f"date: {date}")
    
    # 添加institute
    institute = row.get('institute', '')
    if pd.notna(institute) and str(institute).strip():
        info_parts.append(f"institute: {institute}")
    
    # 用换行符连接
    return '\n'.join(info_parts)

def add_info_field(input_file, output_file):
    """添加info字段到Excel"""
    # 读取Excel
    df = pd.read_excel(input_file)
    
    # 检查必要字段
    required_fields = ['abstract', 'date', 'institute']
    missing_fields = [f for f in required_fields if f not in df.columns]
    
    if missing_fields:
        print(f"警告: 缺少以下字段: {', '.join(missing_fields)}")
        print("将跳过这些字段处理")
    
    # 创建info字段
    df['info'] = df.apply(create_info_field, axis=1)
    
    # 保存结果
    df.to_excel(output_file, index=False)
    
    print(f"处理完成！")
    print(f"总行数: {len(df)}")
    print(f"已添加info字段")
    print(f"输出文件: {output_file}")
    
    # 显示前几行示例
    if len(df) > 0:
        print("\n示例（前3行的info字段）:")
        print("-" * 50)
        for idx, info in enumerate(df['info'].head(3), 1):
            print(f"\n第{idx}行:")
            print(info[:200] + "..." if len(str(info)) > 200 else info)


input_file = "output/清洗后元数据.xlsx"
output_file = "output/拼接待uie.xlsx"
add_info_field(input_file, output_file)

处理完成！
总行数: 10423
已添加info字段
输出文件: output/拼接待uie.xlsx

示例（前3行的info字段）:
--------------------------------------------------

第1行:
abstract: 为探究4种常见果树核桃Juglans regia、桃Amygdalus persica、石榴Punica granatum、柿Diospyros kaki叶片及入侵植物三叶鬼针草Bidens pilosa植株地上部分对三叶鬼针草的化感作用,采用生物测定法,研究4种果树叶片及三叶鬼针草植株浸提液在4个质量浓度0、0.005、0.025、0.05g·mL-1梯度下对三叶鬼针草种子萌...

第2行:
abstract: 三叶鬼针草是重要的旱地杂草，具有较强的化感作用，在其生长多的地方玉米长势差。为探讨三叶鬼针草对玉米的化感作用，采用盆栽试验，用浓度为0、0.005、0.010、0.020、0.040 gmL的三叶鬼针草叶水提液对玉米种子进行处理，观察其对玉米幼苗出土（出苗率、出苗势、出苗速率）及生长（根长、苗高、苗重）的影响。结果表明，三叶鬼针草叶的水提液对玉米幼苗的出土及生长有极显著或显著影...

第3行:
abstract: 采用无水乙醇作为溶剂,从三叶鬼针草根、茎、叶及全草提取活性物质,并对得到的三叶鬼针草提取物进行体外抗菌及抗氧化试验。结果表明:三叶鬼针草的根、茎、叶及全草乙醇提取物的提取率分别为5.90、8.10、11.40、6.90。三叶鬼针草不同部位的乙醇提取物均未呈现良好的抗菌活性,其中,仅叶提取物对枯草芽孢杆菌有轻微的抑制效果,抑菌圈为7.92mm。三叶鬼针草根、茎、叶等部位提取物均具...


In [1]:
import json
from openai import OpenAI

client = OpenAI(
    api_key="<your api key>",
    base_url="https://api.deepseek.com",
)

system_prompt = """
请阅读以下段落，提取其中提到的入侵物种及其对应的发现地点、发现时间。如果地点是一个范围，请标注为区域；如果是一个具体点，请标注为采样点。相同地点的不同植物可以归类，以列表返回。参考以下json格式输出：
{
  "species": ["野燕麦"],
  "location": [
    {"name": "河西地区", "type": "区域"},
    {"name": "武威广场", "type": "具体点"},
    {"name": "玉门镇", "type": "具体点"}
  ],
  "time": ["1984年"]
}
"""

user_prompt = "为掌握北京地区湿地维管束植物外来种的多样性特征及群落构成和分布,采用样带和样方相结合的方法,参考相关资料,从北京市湿地植物中筛选出归化种和入侵种,分析其数量和出现频率;计算外来种的重要值,分析其群落组成;根据实地调查数据分析外来种的群落分布特征。共筛选出外来种19种,其中通奶草、豆瓣菜、野牛草等6种植物为归化种,反枝苋、小蓬草、豚草等13种植物对湿地生境、农作物或当地人畜造成危害,为入侵种;外来种中湿生或水生植物所占比例小,说明了水资源缺乏导致的湿地退化是生物入侵的重要原因之一。最后,对入侵种造成的危害及防治方法进行了论述。"

messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    response_format={
        'type': 'json_object'
    }
)

print(json.loads(response.choices[0].message.content))

AuthenticationError: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****key> is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}

In [ ]:
import pandas as pd
import json
import time
from openai import OpenAI
from tqdm import tqdm

# ================= 配置区 =================
API_KEY = "<API_KEY>"
BASE_URL = "https://api.deepseek.com"
INPUT_FILE = "data/output_combined.xlsx"
OUTPUT_FILE = "uie_results_temp.xlsx"
CONTENT_COLUMN = "摘要"  
FILTER_COLUMN = "判断结果"
FILTER_VALUE = "判断为1"
# ==========================================

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

SYSTEM_PROMPT = """
请阅读以下段落，提取其中提到的入侵物种及其对应的发现地点、发现时间。
如果地点是一个范围(省份/地理区域)，请标注为区域；如果是一个具体点，请标注为采样点。
相同地点的不同植物可以归类，以列表返回。必须严格参考以下json格式输出：
{
  "species": ["野燕麦"],
  "location": [
    {"name": "河西地区", "type": "区域"},
    {"name": "武威广场", "type": "具体点"},
    {"name": "玉门镇", "type": "具体点"}
  ],
  "time": ["1984年"]
}
"""

def extract_info(text):
    """调用 API 提取信息"""
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": text}
            ],
            response_format={'type': 'json_object'},
            timeout=30  # 设置超时
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API调用失败: {e}")
        return None



In [2]:
def main():
    print(f"正在读取文件: {INPUT_FILE}...")
    df = pd.read_excel(INPUT_FILE)

    target_df = df[df[FILTER_COLUMN] == FILTER_VALUE].copy()
    print(f"筛选完成，共有 {len(target_df)} 行待处理。")

    results = []

    for index, row in tqdm(target_df.iterrows(), total=len(target_df), desc="提取进度"):
        if index > 300:
            break
        
        text_content = str(row[CONTENT_COLUMN])
        
        json_str = extract_info(text_content)
        
        if json_str:
            try:
                # 解析验证一下是否为有效 JSON
                parsed_json = json.loads(json_str)
                # 将解析后的各个部分存入新列（可选）
                results.append({
                    "raw_index": index,
                    "extracted_json": json_str,
                    "species": ", ".join(parsed_json.get("species", [])),
                    "locations": str(parsed_json.get("location", [])),
                    "times": ", ".join(parsed_json.get("time", []))
                })
            except:
                results.append({"raw_index": index, "extracted_json": "JSON解析失败", "species": "", "locations": "", "times": ""})
        else:
            results.append({"raw_index": index, "extracted_json": "API无响应", "species": "", "locations": "", "times": ""})

        # 每隔 10 行保存一次临时文件，防止程序崩溃丢失数据
        if len(results) % 10 == 0:
            temp_res_df = pd.DataFrame(results)
            temp_res_df.to_excel(f"temp_results_backup{index}.xlsx", index=False)

    # 5. 合并结果并保存
    res_df = pd.DataFrame(results)
    # 将提取出的信息与原数据合并（按索引对齐）
    final_df = target_df.reset_index().merge(res_df, left_on="index", right_on="raw_index", how="left")
    
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\n处理完成！结果已保存至: {OUTPUT_FILE}")


In [4]:
main()

正在读取文件: data/output_combined.xlsx...
筛选完成，共有 5251 行待处理。


提取进度:   0%|          | 22/5251 [01:23<5:29:50,  3.78s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import json
import time
from openai import OpenAI
from tqdm import tqdm

# ================= 配置区 =================
API_KEY = "<API_KEY>"
BASE_URL = "https://api.deepseek.com"
INPUT_FILE = "output/拼接待uie.xlsx"
OUTPUT_FILE = "uie结果完整版.xlsx"
CONTENT_COLUMN = "info"  
FILTER_COLUMN = "relationship"
FILTER_VALUE = "相关"
# ==========================================

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

SYSTEM_PROMPT = """
请阅读以下abstract，提取其中提到的入侵物种及其对应的发现地点、发现时间，若未明确提及地点和时间，根据所给date和institute信息进行提取；
根据给出的非结构化地理信息，进行推理或者使用常识将地址转换为省-市-区的结构化地址；
相同地点的不同植物可以归类，以列表返回；
时间信息以yyyy-mm形式给出。必须严格参考以下json格式输出：
{
  "species": ["野燕麦"],
  "location": {"extra_addr": "闽江河口鳝鱼滩裸滩湿地", "struc_addr": "福建省-福州市-长乐区"}
  "time": ["1984-10"]
}
"""

def extract_info(text):
    """调用 API 提取信息"""
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": text}
            ],
            response_format={'type': 'json_object'},
            timeout=30  # 设置超时
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API调用失败: {e}")
        return None

def main():
    print(f"正在读取文件: {INPUT_FILE}...")
    df = pd.read_excel(INPUT_FILE)

    target_df = df[df[FILTER_COLUMN] == FILTER_VALUE].copy()
    print(f"筛选完成，共有 {len(target_df)} 行待处理。")

    results = []

    for index, row in tqdm(target_df.iterrows(), total=len(target_df), desc="提取进度"):
        if index <= 5826:
            continue
        
        text_content = str(row[CONTENT_COLUMN])
        
        json_str = extract_info(text_content)
        
        if json_str:
            try:
                # 解析验证一下是否为有效 JSON
                parsed_json = json.loads(json_str)
                # 将解析后的各个部分存入新列（可选）
                results.append({
                    "raw_index": index,
                    "extracted_json": json_str,
                    "species": ", ".join(parsed_json.get("species", [])),
                    "locations": str(parsed_json.get("location", {})),
                    "times": ", ".join(parsed_json.get("time", []))
                })
            except:
                results.append({"raw_index": index, "extracted_json": "JSON解析失败", "species": "", "locations": "", "times": ""})
        else:
            results.append({"raw_index": index, "extracted_json": "API无响应", "species": "", "locations": "", "times": ""})

        # 每隔 10 行保存一次临时文件，防止程序崩溃丢失数据
        if len(results) % 10 == 0:
            temp_res_df = pd.DataFrame(results)
            temp_res_df.to_excel(f"temp_results_backup{index}.xlsx", index=False)

    # 5. 合并结果并保存
    res_df = pd.DataFrame(results)
    # 将提取出的信息与原数据合并（按索引对齐）
    final_df = target_df.reset_index().merge(res_df, left_on="index", right_on="raw_index", how="left")
    
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\n处理完成！结果已保存至: {OUTPUT_FILE}")

main()

正在读取文件: output/拼接待uie.xlsx...
筛选完成，共有 5900 行待处理。


提取进度: 100%|██████████| 5900/5900 [2:12:40<00:00,  1.35s/it]  



处理完成！结果已保存至: uie结果完整版.xlsx
